# Economic Impact & Market Dynamics Analysis

This notebook provides comprehensive economic impact modeling and market dynamics analysis of Alaska OCS oil and gas leases. We examine:

1. **Economic Impact Assessment** - Total economic impact, job creation, revenue analysis
2. **Market Dynamics & Competition** - Bidding behavior, market concentration, strategic patterns
3. **Financial Risk Modeling** - Monte Carlo simulation, VaR calculations, break-even analysis
4. **Policy Impact Quantification** - Economic impact of regulations, revenue optimization

## Table of Contents
- [Setup and Data Loading](#setup)
- [Economic Impact Assessment](#economic-impact)
- [Market Dynamics Analysis](#market-dynamics)
- [Financial Risk Modeling](#financial-risk)
- [Policy Impact Analysis](#policy-impact)
- [Summary and Recommendations](#summary)

## Setup and Data Loading {#setup}

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
from scipy import stats
from scipy.optimize import minimize
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import datetime as dt
from collections import Counter

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("📊 Economic Impact & Market Dynamics Analysis")
print("=" * 50)
print(f"Analysis Date: {dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("Libraries loaded successfully!")

In [ ]:
# Load the lease data
df = pd.read_csv('../data/AK_Leases.csv')

# Data preprocessing for economic analysis
df['lease_date'] = pd.to_datetime(df['lease_date'], errors='coerce')
df['year'] = df['lease_date'].dt.year
df['decade'] = (df['year'] // 10) * 10

# Clean and prepare financial data
financial_cols = ['high_bid_dlrs', 'royalty_rate', 'rental_rate']
for col in financial_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Create economic impact variables
df['bid_per_acre'] = df['high_bid_dlrs'] / df['lease_size_acres'].replace(0, np.nan)
df['total_lease_value'] = df['high_bid_dlrs'] + (df['rental_rate'] * df['lease_size_acres'] * 5)  # 5-year estimate

print(f"📈 Dataset Overview")
print(f"Total Records: {len(df):,}")
print(f"Date Range: {df['year'].min():.0f} - {df['year'].max():.0f}")
print(f"Total Bid Amount: ${df['high_bid_dlrs'].sum():,.0f}")
print(f"Active Leases: {df[df['lease_status'] == 'Active'].shape[0]:,} ({df[df['lease_status'] == 'Active'].shape[0]/len(df)*100:.1f}%)")

df.head(3)

## Economic Impact Assessment {#economic-impact}

In [ ]:
# Economic Impact Calculations
print("💰 ECONOMIC IMPACT ASSESSMENT")
print("=" * 40)

# Total economic metrics
total_bids = df['high_bid_dlrs'].sum()
total_acres = df['lease_size_acres'].sum()
avg_bid_per_acre = df['bid_per_acre'].mean()
median_lease_value = df['total_lease_value'].median()

# Economic multiplier assumptions (based on industry studies)
DIRECT_MULTIPLIER = 1.0
INDIRECT_MULTIPLIER = 0.8  # Indirect economic activity
INDUCED_MULTIPLIER = 0.6   # Induced spending
TOTAL_MULTIPLIER = DIRECT_MULTIPLIER + INDIRECT_MULTIPLIER + INDUCED_MULTIPLIER

# Calculate economic impacts
direct_impact = total_bids
indirect_impact = total_bids * INDIRECT_MULTIPLIER
induced_impact = total_bids * INDUCED_MULTIPLIER
total_economic_impact = direct_impact * TOTAL_MULTIPLIER

# Job creation estimates (jobs per $1M investment)
JOBS_PER_MILLION = 8.5  # Industry average for oil & gas sector
estimated_jobs = (total_bids / 1_000_000) * JOBS_PER_MILLION

print(f"📊 Total Economic Impact Analysis")
print(f"Direct Impact (Lease Payments): ${direct_impact:,.0f}")
print(f"Indirect Impact (Supply Chain): ${indirect_impact:,.0f}")
print(f"Induced Impact (Spending): ${induced_impact:,.0f}")
print(f"TOTAL ECONOMIC IMPACT: ${total_economic_impact:,.0f}")
print(f"")
print(f"👷 Job Creation Estimates")
print(f"Estimated Jobs Created: {estimated_jobs:,.0f}")
print(f"Jobs per Million Invested: {JOBS_PER_MILLION}")
print(f"")
print(f"📏 Per-Acre Economics")
print(f"Average Bid per Acre: ${avg_bid_per_acre:,.2f}")
print(f"Total Acres Under Lease: {total_acres:,.0f}")
print(f"Median Total Lease Value: ${median_lease_value:,.0f}")

In [ ]:
# Economic Impact by Planning Area
area_impact = df.groupby('planning_area').agg({
    'high_bid_dlrs': ['sum', 'mean', 'count'],
    'lease_size_acres': 'sum',
    'bid_per_acre': 'mean'
}).round(2)

area_impact.columns = ['Total_Bids', 'Avg_Bid', 'Lease_Count', 'Total_Acres', 'Avg_Bid_Per_Acre']
area_impact['Economic_Impact'] = area_impact['Total_Bids'] * TOTAL_MULTIPLIER
area_impact['Est_Jobs'] = (area_impact['Total_Bids'] / 1_000_000) * JOBS_PER_MILLION
area_impact = area_impact.sort_values('Economic_Impact', ascending=False)

print("🗺️ ECONOMIC IMPACT BY PLANNING AREA")
print("=" * 50)
display(area_impact.head(8))

In [ ]:
# Revenue Analysis: Federal vs State
print("💵 REVENUE SHARING ANALYSIS")
print("=" * 35)

# Federal revenue (bonus bids)
federal_bonus_revenue = total_bids

# Estimated production royalties (assuming average production over lease life)
avg_royalty_rate = df['royalty_rate'].mean() / 100  # Convert to decimal
estimated_production_value = total_bids * 5  # Conservative multiplier
estimated_royalty_revenue = estimated_production_value * avg_royalty_rate

# Alaska state share (varies by area, using approximate 50% for federal waters adjacent to state)
STATE_SHARE_BONUS = 0.0  # Federal waters
STATE_SHARE_ROYALTY = 0.27  # Alaska gets 27% of federal royalties in adjacent waters

state_bonus_revenue = federal_bonus_revenue * STATE_SHARE_BONUS
state_royalty_revenue = estimated_royalty_revenue * STATE_SHARE_ROYALTY
federal_royalty_revenue = estimated_royalty_revenue * (1 - STATE_SHARE_ROYALTY)

total_federal_revenue = federal_bonus_revenue + federal_royalty_revenue
total_state_revenue = state_bonus_revenue + state_royalty_revenue

print(f"Federal Government Revenue:")
print(f"  Bonus Payments: ${federal_bonus_revenue:,.0f}")
print(f"  Royalty Share: ${federal_royalty_revenue:,.0f}")
print(f"  TOTAL FEDERAL: ${total_federal_revenue:,.0f}")
print(f"")
print(f"Alaska State Revenue:")
print(f"  Bonus Share: ${state_bonus_revenue:,.0f}")
print(f"  Royalty Share: ${state_royalty_revenue:,.0f}")
print(f"  TOTAL STATE: ${total_state_revenue:,.0f}")
print(f"")
print(f"📈 Revenue Projections")
print(f"Average Royalty Rate: {avg_royalty_rate:.2%}")
print(f"Estimated Production Value: ${estimated_production_value:,.0f}")
print(f"Projected Total Royalties: ${estimated_royalty_revenue:,.0f}")

In [ ]:
# Create economic impact visualization
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Economic Impact by Component', 'Impact by Planning Area', 
                   'Revenue Distribution', 'Annual Economic Activity'),
    specs=[[{"type": "pie"}, {"type": "bar"}],
           [{"type": "pie"}, {"type": "scatter"}]]
)

# Economic impact components
impact_labels = ['Direct Impact', 'Indirect Impact', 'Induced Impact']
impact_values = [direct_impact, indirect_impact, induced_impact]
fig.add_trace(go.Pie(labels=impact_labels, values=impact_values, name="Economic Impact"),
              row=1, col=1)

# Top planning areas by economic impact
top_areas = area_impact.head(6)
fig.add_trace(go.Bar(x=top_areas.index, y=top_areas['Economic_Impact']/1e6, 
                     name="Economic Impact ($M)"),
              row=1, col=2)

# Revenue distribution
revenue_labels = ['Federal Bonus', 'Federal Royalty', 'State Royalty']
revenue_values = [federal_bonus_revenue, federal_royalty_revenue, state_royalty_revenue]
fig.add_trace(go.Pie(labels=revenue_labels, values=revenue_values, name="Revenue"),
              row=2, col=1)

# Annual economic activity
annual_activity = df.groupby('year')['high_bid_dlrs'].sum().reset_index()
annual_activity = annual_activity[annual_activity['year'] >= 1980]  # Focus on modern era
fig.add_trace(go.Scatter(x=annual_activity['year'], 
                        y=annual_activity['high_bid_dlrs']/1e6,
                        mode='lines+markers', name="Annual Bids ($M)"),
              row=2, col=2)

fig.update_layout(height=800, showlegend=True, 
                 title_text="Alaska OCS Economic Impact Analysis")
fig.show()

## Market Dynamics & Competition Analysis {#market-dynamics}

In [ ]:
# Market Concentration Analysis
print("🏢 MARKET DYNAMICS & COMPETITION ANALYSIS")
print("=" * 45)

# Company participation analysis
company_analysis = df.groupby('business_associate_name').agg({
    'high_bid_dlrs': ['sum', 'count', 'mean'],
    'lease_size_acres': 'sum'
}).round(2)

company_analysis.columns = ['Total_Bids', 'Lease_Count', 'Avg_Bid', 'Total_Acres']
company_analysis['Market_Share_Bids'] = (company_analysis['Total_Bids'] / total_bids * 100)
company_analysis['Market_Share_Acres'] = (company_analysis['Total_Acres'] / total_acres * 100)
company_analysis = company_analysis.sort_values('Total_Bids', ascending=False)

# Calculate Herfindahl-Hirschman Index (HHI) for market concentration
market_shares = company_analysis['Market_Share_Bids'].values
hhi = np.sum(market_shares ** 2)

# Market concentration interpretation
if hhi < 1500:
    concentration_level = "Unconcentrated (Competitive)"
elif hhi < 2500:
    concentration_level = "Moderately Concentrated"
else:
    concentration_level = "Highly Concentrated"

print(f"📊 Market Concentration Metrics")
print(f"Herfindahl-Hirschman Index (HHI): {hhi:.1f}")
print(f"Market Concentration Level: {concentration_level}")
print(f"Number of Active Companies: {len(company_analysis)}")
print(f"")
print(f"🔝 Top 10 Companies by Total Bids")
display(company_analysis.head(10)[['Total_Bids', 'Lease_Count', 'Market_Share_Bids', 'Avg_Bid']])

In [ ]:
# Competitive vs Non-Competitive Lease Analysis
print("⚔️ COMPETITIVE BIDDING ANALYSIS")
print("=" * 35)

# Simulate competitive intensity based on bid amounts and market activity
# Higher bids in active periods suggest more competition
df['bid_percentile'] = df['high_bid_dlrs'].rank(pct=True)
df['competitive_intensity'] = np.where(df['bid_percentile'] > 0.75, 'High Competition',
                             np.where(df['bid_percentile'] > 0.5, 'Moderate Competition', 
                                     'Low Competition'))

competition_analysis = df.groupby('competitive_intensity').agg({
    'high_bid_dlrs': ['count', 'sum', 'mean', 'median'],
    'lease_size_acres': 'sum',
    'bid_per_acre': 'mean'
}).round(2)

competition_analysis.columns = ['Lease_Count', 'Total_Bids', 'Mean_Bid', 'Median_Bid', 'Total_Acres', 'Avg_Bid_Per_Acre']

print("📈 Bidding Intensity Analysis")
display(competition_analysis)

# Price discovery analysis
print(f"\n💰 Price Discovery Metrics")
high_comp_premium = competition_analysis.loc['High Competition', 'Mean_Bid'] / competition_analysis.loc['Low Competition', 'Mean_Bid']
print(f"High Competition Premium: {high_comp_premium:.1f}x")
print(f"Bid Range: ${df['high_bid_dlrs'].min():,.0f} - ${df['high_bid_dlrs'].max():,.0f}")
print(f"Coefficient of Variation: {df['high_bid_dlrs'].std() / df['high_bid_dlrs'].mean():.2f}")

In [ ]:
# Strategic Bidding Patterns Analysis
print("🎯 STRATEGIC BIDDING PATTERNS")
print("=" * 32)

# Temporal bidding strategies
temporal_strategy = df.groupby(['business_associate_name', 'decade']).agg({
    'high_bid_dlrs': ['count', 'sum', 'mean'],
    'lease_size_acres': 'sum'
}).round(2)

# Block adjacency analysis (simplified)
# Group by planning area and year to identify clustering behavior
clustering_behavior = df.groupby(['planning_area', 'year', 'business_associate_name']).size().reset_index(name='leases_same_area_year')
avg_clustering = clustering_behavior.groupby('business_associate_name')['leases_same_area_year'].mean().sort_values(ascending=False)

print(f"📍 Geographic Clustering Behavior (Top 10)")
print("Average leases per company per area-year:")
for company, avg_leases in avg_clustering.head(10).items():
    print(f"  {company[:30]:<30}: {avg_leases:.1f}")

# Portfolio optimization indicators
print(f"\n📊 Portfolio Diversification Analysis")
top_companies = company_analysis.head(10).index
for company in top_companies[:5]:
    company_data = df[df['business_associate_name'] == company]
    area_diversity = len(company_data['planning_area'].unique())
    year_span = company_data['year'].max() - company_data['year'].min()
    print(f"{company[:25]:<25}: {area_diversity} areas, {year_span} year span")

In [ ]:
# Market dynamics visualization
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Market Share Distribution', 'Competitive Intensity', 
                   'Bidding Over Time by Top Companies', 'Geographic Competition'),
    specs=[[{"type": "bar"}, {"type": "bar"}],
           [{"type": "scatter"}, {"type": "heatmap"}]]
)

# Top companies market share
top_10_companies = company_analysis.head(10)
fig.add_trace(go.Bar(x=top_10_companies.index[::-1], y=top_10_companies['Market_Share_Bids'][::-1], 
                     orientation='h', name="Market Share %"),
              row=1, col=1)

# Competitive intensity
fig.add_trace(go.Bar(x=competition_analysis.index, y=competition_analysis['Mean_Bid']/1e6,
                     name="Mean Bid ($M)"),
              row=1, col=2)

# Time series for top companies
top_5_companies = company_analysis.head(5).index
for i, company in enumerate(top_5_companies):
    company_time = df[df['business_associate_name'] == company].groupby('year')['high_bid_dlrs'].sum().reset_index()
    if len(company_time) > 0:
        fig.add_trace(go.Scatter(x=company_time['year'], y=company_time['high_bid_dlrs']/1e6,
                                mode='lines+markers', name=company[:15]),
                      row=2, col=1)

# Planning area competition heatmap
area_year_activity = df.pivot_table(values='high_bid_dlrs', index='planning_area', 
                                   columns='decade', aggfunc='sum', fill_value=0)
if len(area_year_activity) > 0:
    fig.add_trace(go.Heatmap(z=area_year_activity.values/1e6, 
                            x=area_year_activity.columns,
                            y=area_year_activity.index,
                            colorscale='Viridis'),
                  row=2, col=2)

fig.update_layout(height=800, showlegend=True,
                 title_text=f"Market Dynamics Analysis (HHI: {hhi:.0f} - {concentration_level})")
fig.show()

## Financial Risk Modeling {#financial-risk}

In [ ]:
# Financial Risk Modeling with Monte Carlo Simulation
print("📈 FINANCIAL RISK MODELING")
print("=" * 30)

# Set random seed for reproducibility
np.random.seed(42)

# Monte Carlo parameters
n_simulations = 10000
oil_price_scenarios = np.array([40, 60, 80, 100, 120])  # $/barrel scenarios
gas_price_scenarios = np.array([2, 3, 4, 5, 6])        # $/MMBtu scenarios

# Historical volatility assumptions (annualized)
oil_volatility = 0.35
gas_volatility = 0.45
correlation = 0.6

# Lease characteristics for modeling
avg_lease_size = df['lease_size_acres'].mean()
avg_bid_amount = df['high_bid_dlrs'].mean()
avg_royalty_rate = df['royalty_rate'].mean() / 100

print(f"🎲 Monte Carlo Simulation Parameters")
print(f"Number of Simulations: {n_simulations:,}")
print(f"Oil Price Volatility: {oil_volatility:.1%}")
print(f"Gas Price Volatility: {gas_volatility:.1%}")
print(f"Price Correlation: {correlation:.2f}")
print(f"Average Lease Size: {avg_lease_size:,.0f} acres")
print(f"Average Bid Amount: ${avg_bid_amount:,.0f}")
print(f"Average Royalty Rate: {avg_royalty_rate:.2%}")

def monte_carlo_lease_value(n_sims, base_oil_price=70, base_gas_price=3.5, lease_life_years=5):
    """Monte Carlo simulation for lease value under different price scenarios"""
    
    # Generate correlated price paths
    # Create correlated random variables
    random_matrix = np.random.multivariate_normal(
        [0, 0], 
        [[1, correlation], [correlation, 1]], 
        n_sims
    )
    
    # Generate price scenarios using geometric Brownian motion
    dt = 1/12  # Monthly steps
    n_steps = lease_life_years * 12
    
    oil_prices = np.zeros((n_sims, n_steps))
    gas_prices = np.zeros((n_sims, n_steps))
    
    oil_prices[:, 0] = base_oil_price
    gas_prices[:, 0] = base_gas_price
    
    for t in range(1, n_steps):
        # Oil price evolution
        oil_drift = -0.5 * oil_volatility**2 * dt
        oil_shock = oil_volatility * np.sqrt(dt) * random_matrix[:, 0]
        oil_prices[:, t] = oil_prices[:, t-1] * np.exp(oil_drift + oil_shock)
        
        # Gas price evolution
        gas_drift = -0.5 * gas_volatility**2 * dt
        gas_shock = gas_volatility * np.sqrt(dt) * random_matrix[:, 1]
        gas_prices[:, t] = gas_prices[:, t-1] * np.exp(gas_drift + gas_shock)
    
    # Calculate production estimates (simplified)
    # Assume production proportional to lease size and price levels
    production_factor = avg_lease_size / 5760  # Normalize by typical lease size
    
    oil_production = production_factor * 1000  # barrels per month
    gas_production = production_factor * 50    # MMBtu per month
    
    # Calculate revenue streams
    monthly_oil_revenue = oil_prices * oil_production
    monthly_gas_revenue = gas_prices * gas_production
    monthly_total_revenue = monthly_oil_revenue + monthly_gas_revenue
    
    # Calculate royalty payments
    monthly_royalties = monthly_total_revenue * avg_royalty_rate
    
    # Calculate net present value
    discount_rate = 0.10  # 10% annual discount rate
    monthly_discount_rate = discount_rate / 12
    
    # Discount factors
    time_periods = np.arange(1, n_steps + 1)
    discount_factors = 1 / (1 + monthly_discount_rate) ** time_periods
    
    # Calculate NPV for each simulation
    npv_royalties = np.sum(monthly_royalties * discount_factors, axis=1)
    total_lease_value = npv_royalties - avg_bid_amount  # Subtract initial investment
    
    return total_lease_value, oil_prices, gas_prices, monthly_royalties

# Run Monte Carlo simulation
print(f"\n🔄 Running Monte Carlo simulation...")
lease_values, oil_price_paths, gas_price_paths, royalty_streams = monte_carlo_lease_value(n_simulations)

print(f"✅ Simulation completed successfully!")

In [ ]:
# Value at Risk (VaR) Analysis
print("⚠️ VALUE AT RISK (VaR) ANALYSIS")
print("=" * 35)

# Calculate VaR at different confidence levels
confidence_levels = [0.95, 0.99, 0.999]
var_results = {}

for conf_level in confidence_levels:
    alpha = 1 - conf_level
    var_value = np.percentile(lease_values, alpha * 100)
    var_results[conf_level] = var_value

# Expected Shortfall (Conditional VaR)
def expected_shortfall(values, confidence_level):
    alpha = 1 - confidence_level
    var = np.percentile(values, alpha * 100)
    return np.mean(values[values <= var])

es_results = {conf_level: expected_shortfall(lease_values, conf_level) 
              for conf_level in confidence_levels}

# Statistical summary
mean_value = np.mean(lease_values)
median_value = np.median(lease_values)
std_value = np.std(lease_values)
min_value = np.min(lease_values)
max_value = np.max(lease_values)

print(f"📊 Lease Value Distribution")
print(f"Mean Lease Value: ${mean_value:,.0f}")
print(f"Median Lease Value: ${median_value:,.0f}")
print(f"Standard Deviation: ${std_value:,.0f}")
print(f"Range: ${min_value:,.0f} to ${max_value:,.0f}")
print(f"")
print(f"📉 Value at Risk (VaR) Results")
for conf_level in confidence_levels:
    print(f"VaR at {conf_level:.1%} confidence: ${var_results[conf_level]:,.0f}")
    print(f"Expected Shortfall at {conf_level:.1%}: ${es_results[conf_level]:,.0f}")
    print(f"")

# Probability of profit
prob_profit = np.mean(lease_values > 0)
prob_loss = 1 - prob_profit

print(f"📈 Profitability Analysis")
print(f"Probability of Profit: {prob_profit:.1%}")
print(f"Probability of Loss: {prob_loss:.1%}")
print(f"Expected Return: ${mean_value:,.0f}")
print(f"Return on Investment: {mean_value/avg_bid_amount:.1%}")

In [ ]:
# Break-even Analysis
print("⚖️ BREAK-EVEN ANALYSIS")
print("=" * 25)

# Calculate break-even oil prices for different scenarios
def calculate_breakeven_price(bid_amount, lease_size, royalty_rate, production_years=5):
    """Calculate break-even oil price given lease parameters"""
    
    # Assumptions for break-even calculation
    monthly_production = (lease_size / 5760) * 1000  # barrels per month
    total_production = monthly_production * 12 * production_years
    
    # Operating costs (simplified)
    operating_cost_per_barrel = 25  # $/barrel
    total_operating_costs = total_production * operating_cost_per_barrel
    
    # Break-even calculation
    # Revenue needed = Bid amount + Operating costs
    required_revenue = bid_amount + total_operating_costs
    
    # Revenue = Oil price * Production * Royalty rate
    # Solving for oil price
    breakeven_price = required_revenue / (total_production * royalty_rate)
    
    return breakeven_price, total_production, total_operating_costs

# Calculate break-even for different lease types
lease_scenarios = {
    'Small Lease': {'bid': 50000, 'size': 2304, 'royalty': 0.125},
    'Average Lease': {'bid': avg_bid_amount, 'size': avg_lease_size, 'royalty': avg_royalty_rate},
    'Large Lease': {'bid': 5000000, 'size': 5760, 'royalty': 0.167},
    'Premium Lease': {'bid': 20000000, 'size': 5760, 'royalty': 0.167}
}

breakeven_results = {}
print(f"🎯 Break-even Oil Price Analysis")
print(f"{'Lease Type':<15} {'Bid Amount':<12} {'Lease Size':<12} {'Break-even':<12} {'Production':<12}")
print(f"{'-'*15} {'-'*12} {'-'*12} {'-'*12} {'-'*12}")

for scenario_name, params in lease_scenarios.items():
    breakeven_price, production, operating_costs = calculate_breakeven_price(
        params['bid'], params['size'], params['royalty']
    )
    breakeven_results[scenario_name] = {
        'breakeven_price': breakeven_price,
        'production': production,
        'operating_costs': operating_costs
    }
    
    print(f"{scenario_name:<15} ${params['bid']:<11,.0f} {params['size']:<12,.0f} ${breakeven_price:<11.0f} {production:<12,.0f}")

print(f"\n💡 Break-even Insights")
avg_breakeven = np.mean([result['breakeven_price'] for result in breakeven_results.values()])
print(f"Average Break-even Price: ${avg_breakeven:.0f}/barrel")
print(f"Current Market Context: WTI ~$80/barrel (as of analysis date)")
viable_leases = sum(1 for result in breakeven_results.values() if result['breakeven_price'] < 80)
print(f"Viable Leases at $80/barrel: {viable_leases}/{len(lease_scenarios)}")

In [ ]:
# Financial risk visualization
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Lease Value Distribution', 'Value at Risk Analysis',
                   'Break-even Prices by Lease Type', 'Risk-Return Profile'),
    specs=[[{"type": "histogram"}, {"type": "bar"}],
           [{"type": "bar"}, {"type": "scatter"}]]
)

# Lease value distribution
fig.add_trace(go.Histogram(x=lease_values/1e6, nbinsx=50, name="Lease Values",
                          histnorm='probability'),
              row=1, col=1)

# VaR comparison
var_labels = [f"{conf:.1%}" for conf in confidence_levels]
var_values = [var_results[conf]/1e6 for conf in confidence_levels]
es_values = [es_results[conf]/1e6 for conf in confidence_levels]

fig.add_trace(go.Bar(x=var_labels, y=var_values, name="Value at Risk"),
              row=1, col=2)
fig.add_trace(go.Bar(x=var_labels, y=es_values, name="Expected Shortfall"),
              row=1, col=2)

# Break-even prices
breakeven_names = list(breakeven_results.keys())
breakeven_prices = [breakeven_results[name]['breakeven_price'] for name in breakeven_names]

fig.add_trace(go.Bar(x=breakeven_names, y=breakeven_prices, name="Break-even Price"),
              row=2, col=1)

# Add market price line
fig.add_hline(y=80, line_dash="dash", line_color="red", 
              annotation_text="Current Market (~$80)", row=2, col=1)

# Risk-return scatter for different lease characteristics
# Sample leases for risk-return analysis
sample_size = min(1000, len(df))
sample_df = df.sample(n=sample_size, random_state=42)

expected_returns = sample_df['high_bid_dlrs'] * 0.15  # Simplified expected return
risk_measures = sample_df['bid_per_acre']  # Use bid per acre as risk proxy

fig.add_trace(go.Scatter(x=risk_measures, y=expected_returns/1e6,
                        mode='markers', name="Lease Risk-Return",
                        text=sample_df['planning_area']),
              row=2, col=2)

fig.update_layout(height=800, showlegend=True,
                 title_text=f"Financial Risk Analysis (Prob. Profit: {prob_profit:.1%})")

# Update axis labels
fig.update_xaxes(title_text="Lease Value ($M)", row=1, col=1)
fig.update_xaxes(title_text="Confidence Level", row=1, col=2)
fig.update_xaxes(title_text="Lease Type", row=2, col=1)
fig.update_xaxes(title_text="Risk ($/acre)", row=2, col=2)

fig.update_yaxes(title_text="Probability", row=1, col=1)
fig.update_yaxes(title_text="Value ($M)", row=1, col=2)
fig.update_yaxes(title_text="Price ($/barrel)", row=2, col=1)
fig.update_yaxes(title_text="Expected Return ($M)", row=2, col=2)

fig.show()

## Policy Impact Quantification {#policy-impact}

In [ ]:
# Policy Impact Analysis
print("🏛️ POLICY IMPACT QUANTIFICATION")
print("=" * 35)

# Identify policy periods based on major regulatory changes
policy_periods = {
    'Early Era (1976-1989)': (1976, 1989),
    'OCSLA Amendment (1990-1999)': (1990, 1999),
    'Modern Era (2000-2009)': (2000, 2009),
    'Obama Policies (2010-2016)': (2010, 2016),
    'Recent Era (2017-2024)': (2017, 2024)
}

# Calculate metrics for each policy period
policy_analysis = {}

for period_name, (start_year, end_year) in policy_periods.items():
    period_data = df[(df['year'] >= start_year) & (df['year'] <= end_year)]
    
    if len(period_data) > 0:
        policy_analysis[period_name] = {
            'lease_count': len(period_data),
            'total_bids': period_data['high_bid_dlrs'].sum(),
            'avg_bid': period_data['high_bid_dlrs'].mean(),
            'median_bid': period_data['high_bid_dlrs'].median(),
            'total_acres': period_data['lease_size_acres'].sum(),
            'avg_royalty': period_data['royalty_rate'].mean(),
            'active_rate': (period_data['lease_status'] == 'Active').mean() * 100,
            'years_span': end_year - start_year + 1,
            'annual_avg_bids': period_data['high_bid_dlrs'].sum() / (end_year - start_year + 1)
        }

# Convert to DataFrame for easier analysis
policy_df = pd.DataFrame(policy_analysis).T
policy_df = policy_df.round(2)

print(f"📊 Policy Period Analysis")
display(policy_df[['lease_count', 'total_bids', 'avg_bid', 'total_acres', 'annual_avg_bids']])

print(f"\n📈 Policy Impact Metrics")
print(f"Highest Activity Period: {policy_df['annual_avg_bids'].idxmax()}")
print(f"Peak Annual Average: ${policy_df['annual_avg_bids'].max():,.0f}")
print(f"Lowest Activity Period: {policy_df['annual_avg_bids'].idxmin()}")
print(f"Minimum Annual Average: ${policy_df['annual_avg_bids'].min():,.0f}")

# Calculate policy impact ratios
baseline_period = 'Early Era (1976-1989)'
if baseline_period in policy_df.index:
    baseline_activity = policy_df.loc[baseline_period, 'annual_avg_bids']
    print(f"\n🔄 Policy Impact Ratios (vs {baseline_period})")
    for period in policy_df.index:
        if period != baseline_period:
            ratio = policy_df.loc[period, 'annual_avg_bids'] / baseline_activity
            print(f"{period}: {ratio:.1f}x baseline activity")

In [ ]:
# Economic Impact of Moratorium Periods
print("🚫 MORATORIUM IMPACT ANALYSIS")
print("=" * 32)

# Identify low-activity periods (potential moratoriums or reduced activity)
annual_activity = df.groupby('year').agg({
    'high_bid_dlrs': 'sum',
    'lease_size_acres': 'sum',
    'business_associate_name': 'nunique'
}).reset_index()

annual_activity['annual_bids_millions'] = annual_activity['high_bid_dlrs'] / 1e6

# Define moratorium threshold (years with <10% of peak activity)
peak_activity = annual_activity['high_bid_dlrs'].max()
moratorium_threshold = peak_activity * 0.1

low_activity_years = annual_activity[annual_activity['high_bid_dlrs'] < moratorium_threshold]['year'].tolist()
high_activity_years = annual_activity[annual_activity['high_bid_dlrs'] >= moratorium_threshold]['year'].tolist()

# Calculate economic impact of reduced activity
avg_high_activity = annual_activity[annual_activity['high_bid_dlrs'] >= moratorium_threshold]['high_bid_dlrs'].mean()
avg_low_activity = annual_activity[annual_activity['high_bid_dlrs'] < moratorium_threshold]['high_bid_dlrs'].mean()

annual_opportunity_cost = avg_high_activity - avg_low_activity
total_low_activity_years = len(low_activity_years)
total_opportunity_cost = annual_opportunity_cost * total_low_activity_years

print(f"📉 Moratorium Period Analysis")
print(f"Peak Annual Activity: ${peak_activity:,.0f}")
print(f"Moratorium Threshold (<10% of peak): ${moratorium_threshold:,.0f}")
print(f"Low Activity Years: {len(low_activity_years)} ({low_activity_years[:10]}...)")
print(f"High Activity Years: {len(high_activity_years)}")
print(f"")
print(f"💰 Economic Impact")
print(f"Average High Activity Year: ${avg_high_activity:,.0f}")
print(f"Average Low Activity Year: ${avg_low_activity:,.0f}")
print(f"Annual Opportunity Cost: ${annual_opportunity_cost:,.0f}")
print(f"Total Estimated Opportunity Cost: ${total_opportunity_cost:,.0f}")
print(f"Total Economic Impact Lost: ${total_opportunity_cost * TOTAL_MULTIPLIER:,.0f}")

# Jobs impact
jobs_impact = (total_opportunity_cost / 1_000_000) * JOBS_PER_MILLION
print(f"Estimated Jobs Impact: {jobs_impact:,.0f} jobs")

In [ ]:
# Regulatory Cost-Benefit Analysis
print("⚖️ REGULATORY COST-BENEFIT ANALYSIS")
print("=" * 38)

# Environmental regulation scenarios
regulation_scenarios = {
    'Baseline (Current)': {'environmental_cost': 0, 'delay_months': 0, 'success_rate': 0.15},
    'Enhanced Environmental': {'environmental_cost': 50000, 'delay_months': 6, 'success_rate': 0.12},
    'Streamlined Process': {'environmental_cost': 25000, 'delay_months': -3, 'success_rate': 0.18},
    'Maximum Protection': {'environmental_cost': 150000, 'delay_months': 12, 'success_rate': 0.08}
}

# Calculate net present value impact for each scenario
discount_rate_annual = 0.08

regulation_impact = {}

for scenario, params in regulation_scenarios.items():
    # Calculate impact on lease economics
    additional_costs = params['environmental_cost']
    time_delay_impact = (params['delay_months'] / 12) * discount_rate_annual * avg_bid_amount
    success_rate_multiplier = params['success_rate'] / regulation_scenarios['Baseline (Current)']['success_rate']
    
    # Adjusted lease value
    adjusted_lease_value = (avg_bid_amount - additional_costs - time_delay_impact) * success_rate_multiplier
    
    # Environmental benefits (monetized - simplified)
    environmental_benefit_per_lease = params['environmental_cost'] * 2  # Assume 2:1 benefit ratio
    
    # Net social value
    net_social_value = adjusted_lease_value + environmental_benefit_per_lease - additional_costs
    
    regulation_impact[scenario] = {
        'adjusted_lease_value': adjusted_lease_value,
        'environmental_benefit': environmental_benefit_per_lease,
        'additional_costs': additional_costs,
        'time_delay_cost': time_delay_impact,
        'net_social_value': net_social_value,
        'success_rate': params['success_rate']
    }

# Convert to DataFrame
regulation_df = pd.DataFrame(regulation_impact).T
regulation_df = regulation_df.round(0)

print(f"📊 Regulatory Scenario Analysis (Per Average Lease)")
display(regulation_df[['adjusted_lease_value', 'environmental_benefit', 'additional_costs', 'net_social_value']])

# Optimal policy recommendation
optimal_scenario = regulation_df['net_social_value'].idxmax()
print(f"\n🎯 Policy Recommendation")
print(f"Optimal Scenario: {optimal_scenario}")
print(f"Net Social Value: ${regulation_df.loc[optimal_scenario, 'net_social_value']:,.0f}")
print(f"Success Rate: {regulation_df.loc[optimal_scenario, 'success_rate']:.1%}")

# Calculate aggregate impact across all historical leases
total_leases = len(df)
aggregate_impact = regulation_df.loc[optimal_scenario, 'net_social_value'] * total_leases
print(f"\n🌍 Aggregate Impact Estimate")
print(f"Total Historical Leases: {total_leases:,}")
print(f"Aggregate Net Social Value: ${aggregate_impact:,.0f}")
print(f"Value vs Current Policy: ${(regulation_df.loc[optimal_scenario, 'net_social_value'] - regulation_df.loc['Baseline (Current)', 'net_social_value']) * total_leases:,.0f}")

In [ ]:
# Revenue Optimization Model
print("📈 REVENUE OPTIMIZATION MODEL")
print("=" * 32)

# Optimal timing model for lease sales
# Based on oil price cycles and market conditions

# Historical oil price approximations (simplified)
oil_price_history = {
    1980: 35, 1985: 28, 1990: 24, 1995: 18, 2000: 30, 
    2005: 56, 2010: 80, 2015: 48, 2020: 40, 2024: 80
}

# Match lease years with approximate oil prices
df_price = df.copy()
df_price['oil_price_estimate'] = df_price['year'].map(
    lambda x: min(oil_price_history.keys(), key=lambda k: abs(k-x)) if not pd.isna(x) else None
).map(oil_price_history)

# Analyze bid sensitivity to oil prices
price_correlation = df_price[['high_bid_dlrs', 'oil_price_estimate']].corr().iloc[0,1]

# Optimal timing analysis
price_bins = [0, 30, 50, 70, 100, 150]
price_labels = ['<$30', '$30-50', '$50-70', '$70-100', '>$100']
df_price['price_bin'] = pd.cut(df_price['oil_price_estimate'], bins=price_bins, labels=price_labels, include_lowest=True)

price_analysis = df_price.groupby('price_bin').agg({
    'high_bid_dlrs': ['count', 'sum', 'mean', 'median'],
    'lease_size_acres': 'sum'
}).round(0)

price_analysis.columns = ['Lease_Count', 'Total_Bids', 'Mean_Bid', 'Median_Bid', 'Total_Acres']
price_analysis['Revenue_per_Acre'] = price_analysis['Total_Bids'] / price_analysis['Total_Acres']

print(f"🛢️ Oil Price Impact on Lease Revenue")
print(f"Price-Bid Correlation: {price_correlation:.3f}")
print(f"")
display(price_analysis)

# Revenue optimization recommendations
optimal_price_range = price_analysis['Mean_Bid'].idxmax()
revenue_multiplier = price_analysis.loc[optimal_price_range, 'Mean_Bid'] / price_analysis['Mean_Bid'].min()

print(f"\n🎯 Revenue Optimization Insights")
print(f"Optimal Price Range for Lease Sales: {optimal_price_range}")
print(f"Revenue Multiplier vs Worst Timing: {revenue_multiplier:.1f}x")
print(f"Average Bid in Optimal Range: ${price_analysis.loc[optimal_price_range, 'Mean_Bid']:,.0f}")

# Future scenario modeling
future_scenarios = {'Conservative': 60, 'Moderate': 80, 'Optimistic': 100}
print(f"\n🔮 Future Revenue Projections (Next Lease Sale)")
baseline_bid = df['high_bid_dlrs'].mean()
for scenario, oil_price in future_scenarios.items():
    # Simple linear relationship assumption
    price_factor = oil_price / 70  # Normalize to $70 baseline
    projected_avg_bid = baseline_bid * price_factor
    
    # Assume 100 leases in next sale
    projected_revenue = projected_avg_bid * 100
    projected_economic_impact = projected_revenue * TOTAL_MULTIPLIER
    
    print(f"{scenario} (${oil_price}/barrel): ${projected_revenue:,.0f} revenue, ${projected_economic_impact:,.0f} economic impact")

In [ ]:
# Policy impact visualization
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Policy Period Comparison', 'Annual Activity vs Oil Prices',
                   'Regulatory Cost-Benefit', 'Revenue Optimization by Price Range'),
    specs=[[{"type": "bar"}, {"type": "scatter"}],
           [{"type": "bar"}, {"type": "bar"}]]
)

# Policy period comparison
fig.add_trace(go.Bar(x=policy_df.index, y=policy_df['annual_avg_bids']/1e6,
                     name="Annual Avg Bids ($M)"),
              row=1, col=1)

# Annual activity vs oil prices
annual_with_price = annual_activity.merge(
    pd.DataFrame(list(oil_price_history.items()), columns=['year', 'oil_price']),
    on='year', how='left'
).dropna()

if len(annual_with_price) > 0:
    fig.add_trace(go.Scatter(x=annual_with_price['oil_price'], 
                            y=annual_with_price['annual_bids_millions'],
                            mode='markers+text', name="Annual Activity",
                            text=annual_with_price['year'],
                            textposition="top center"),
                  row=1, col=2)

# Regulatory scenarios
reg_scenarios = list(regulation_df.index)
net_values = regulation_df['net_social_value'].values / 1000  # Convert to thousands

colors = ['red' if x < 0 else 'green' for x in net_values]
fig.add_trace(go.Bar(x=reg_scenarios, y=net_values, name="Net Social Value ($K)",
                     marker_color=colors),
              row=2, col=1)

# Revenue optimization
if not price_analysis.empty:
    fig.add_trace(go.Bar(x=price_analysis.index, y=price_analysis['Mean_Bid']/1e6,
                         name="Mean Bid ($M)"),
                  row=2, col=2)

fig.update_layout(height=800, showlegend=True,
                 title_text="Policy Impact & Revenue Optimization Analysis")

# Update axis labels
fig.update_xaxes(title_text="Policy Period", row=1, col=1)
fig.update_xaxes(title_text="Oil Price ($/barrel)", row=1, col=2)
fig.update_xaxes(title_text="Regulatory Scenario", row=2, col=1)
fig.update_xaxes(title_text="Oil Price Range", row=2, col=2)

fig.update_yaxes(title_text="Annual Avg ($M)", row=1, col=1)
fig.update_yaxes(title_text="Annual Bids ($M)", row=1, col=2)
fig.update_yaxes(title_text="Net Value ($K)", row=2, col=1)
fig.update_yaxes(title_text="Mean Bid ($M)", row=2, col=2)

fig.show()

## Summary and Recommendations {#summary}

In [ ]:
# Executive Summary
print("📋 EXECUTIVE SUMMARY & RECOMMENDATIONS")
print("=" * 42)

print(f"💰 ECONOMIC IMPACT HIGHLIGHTS")
print(f"• Total Economic Impact: ${total_economic_impact:,.0f}")
print(f"• Job Creation Estimate: {estimated_jobs:,.0f} jobs")
print(f"• Federal Revenue: ${total_federal_revenue:,.0f}")
print(f"• Alaska State Revenue: ${total_state_revenue:,.0f}")
print(f"")

print(f"🏢 MARKET DYNAMICS INSIGHTS")
print(f"• Market Concentration (HHI): {hhi:.0f} - {concentration_level}")
print(f"• High Competition Premium: {high_comp_premium:.1f}x")
print(f"• Active Companies: {len(company_analysis)}")
print(f"")

print(f"⚠️ FINANCIAL RISK ASSESSMENT")
print(f"• Probability of Profit: {prob_profit:.1%}")
print(f"• Average Break-even Price: ${avg_breakeven:.0f}/barrel")
print(f"• VaR (95% confidence): ${var_results[0.95]:,.0f}")
print(f"• Expected Return: ${mean_value:,.0f}")
print(f"")

print(f"🏛️ POLICY IMPACT FINDINGS")
print(f"• Optimal Policy Scenario: {optimal_scenario}")
print(f"• Opportunity Cost of Low Activity: ${total_opportunity_cost:,.0f}")
print(f"• Optimal Lease Sale Timing: {optimal_price_range} oil prices")
print(f"")

print(f"🎯 STRATEGIC RECOMMENDATIONS")
print(f"")
print(f"1. REVENUE OPTIMIZATION")
print(f"   • Time lease sales during {optimal_price_range} oil price periods")
print(f"   • Potential {revenue_multiplier:.1f}x revenue increase with optimal timing")
print(f"   • Focus on high-competition areas for premium pricing")
print(f"")
print(f"2. RISK MANAGEMENT")
print(f"   • {100-prob_profit*100:.1f}% of leases show negative returns")
print(f"   • Implement break-even analysis at ${avg_breakeven:.0f}/barrel threshold")
print(f"   • Consider portfolio diversification across planning areas")
print(f"")
print(f"3. POLICY FRAMEWORK")
print(f"   • {optimal_scenario.lower()} provides optimal social value")
print(f"   • Balance environmental protection with economic activity")
print(f"   • Reduce regulatory uncertainty to maintain market participation")
print(f"")
print(f"4. MARKET DEVELOPMENT")
print(f"   • Current {concentration_level.lower()} market structure is healthy")
print(f"   • Encourage new entrants to maintain competition")
print(f"   • Support strategic clustering for operational efficiency")

print(f"\n" + "=" * 50)
print(f"Analysis completed: {dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Total computation time: Monte Carlo with {n_simulations:,} simulations")
print(f"Dataset coverage: {len(df):,} leases from {df['year'].min():.0f}-{df['year'].max():.0f}")